# sample generator
This notebook is for creating the random sample .txt files for the experiment.

In [1]:
import pandas as pd
import numpy as np

 ## Contents
 1. [Create separate file with contexts](#Create-separate-file-with-contexts)
 2. [Create separate file without contexts](#Create-separate-file-without-contexts)
 3. [Creating the files for the experiment](#Creating-files-for-the-experiment)
 4. [Automate Paraphrase Generator](#Automate-pharaphrase-generator)


In [2]:
# import the database file from the TGrep2 searching
df = pd.read_csv("../results/swbd.tab", sep='\t', engine='python')

In [3]:
# This makes the display show more info
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Creating the files for the experiment

## Constrain dataset
for experimental mock-up

First we have to remove the questions that we don;t want to include:
1. non-embedded or root questions
2. no degree questions
3. no identity questions
4. generally only monomorphemic wh-phrases
5. only who-, where-, and how-questions

In [4]:
df_root = df[df.QuestionType=="root"]
df_emb = df[df.QuestionType=="embedded"]

In [5]:
critical = df[(df['QuestionType'] == 'root') # only root questions
              & 
              (df['DegreeQ'] == 'no' ) # no degree questions
              &
              (df['IdentityQ'] == "no") # no identity questions
              &
              (df['Wh'].isin(['how','How','where','Where','who','Who'])) # just these wh-words
              &
              (df['WhPhaseType'] == 'monomorphemic') # monomorphic wh only (might get anything not go by degQ)
             ]

In [6]:
(len(df_root)+len(df_emb))/len(df)*100

46.44572997352682

In [7]:
# Percentage of total dataset
len(critical)/len(df_root)*100

19.488074461896453

In [8]:
len(critical)/(len(df_root)+len(df_emb))*100

7.071986489339244

In [9]:
len(critical)/len(df)*100

3.2846357486028044

In [82]:
len(critical)

335

Notes from Judith meeting 11/5:
15 how , 10 where, 5 who

randomly divide the gru
each list should get about 30 judgements

but pilot first with 10 items to test different dependent measures


In [11]:
critical.pivot_table(index=['Wh'], values="Sentence", aggfunc=len).groupby(["Wh"]).Sentence.transform(lambda x: x/len(critical)).reset_index()

,Wh,Sentence
0,how,0.549254
1,where,0.289552
2,who,0.161194


In [12]:
40/335

0.11940298507462686

should contain about 3 modal in each batch
10 lists of 30
1 list of 35

balance so that there are 3 modals 


QUESTION PHRASING
avoid using 'paraphrase'
1. based on the question in red, which of the following does the speaker want to know?
2. "none of the above"
3. ellipsis before the 
4. practice examples that are fake from the literature clear MS/MA answers....
4. get rid of trace

should be a way to highlight just the question, and not the entire sentence
"how can you tell where it's going to come down at"


is there a way to reorder the elements of the question to create the paraphrase
strip the "do" and insert "that"


rhetorical questions: "so my family may say that on occasion, who knows"

Piloting
10 items that will be repiloted for each different testing 

vary between subjects the order of the paraphrases
sample randomly at the beginning of the experiment what the order is
but keep it consistent in the experiment and change between subjects


By monday post on proliferate*



look at the lab wifi for pre-registration on OSF

get R going....

In [10]:
len(critical)

335

## Paraphrase Generator
this should take as input the entire constrained dataframe from the above section, and then generate the paraphrases

For Who questions: Who is a person...? / Who is some person...? / "Who is every person..." / "Who is the person..."

In [13]:
# read in df with contexts
cntxts = pd.read_csv("swbd_contexts.csv")

In [14]:
cntxts = cntxts.drop(columns="FollowingContext")

In [15]:
# get the indixes from critical
crit_index = critical.Item_ID

In [22]:
# subset to the items that are just the ones filtered in the previos section

# otherwise, if using the database file with contexts directly in there, then this step
# is not necessary
df_valid = cntxts[cntxts["TGrepID"].isin(set(crit_index))]

In [23]:
who = df_valid[df_valid["EntireSentence"].str.contains("Who|who")]
where = df_valid[df_valid["EntireSentence"].str.contains("Where|where")]
how = df_valid[df_valid["EntireSentence"].str.contains("How|how")]

In [30]:
df_valid.head()

,TGrepID,EntireSentence,PreceedingContext
95,1721:4,how do you feel *t*-1 about rap music?,"###there's such a wide selection,### i think 0 i like a lot, i like a little bit of a lot of different types of music.###you know, i, i, i like music that *t*-1 is, that i feel###if it is performed *-1 correctly or if it's done *-2 right, or if the version is done *-3 right, i like it,###speakera15.###yeah.###speakerb16.###but, if, if it's not *?*, then i won't *?*.### i, i really don't,###speakera17."
189,2936:24,"oh, rea-, where *ich*-2 did you go to school in indiana *t*-1?","###speakerb58.###so, that, uh, you, you became accustomed to it, i guess.###speakera59.###uh-huh.###speakerb60.###but, uh, otherwise w-, as i said, we have had, uh, a relatively mild winter, * speaking for this area of the country.###speakera61.###uh-huh.###speakerb62.###speakera63."
246,4134:4,"how did you hear,","###do you work for t i?###speakera90.###no,###no,###i work for g t e. in maryland,###speakerb91.###really?###speakera92.###yes.###speakerb93."
247,4140:4,how did i hear about it *t*-1?,"###really?###speakera92.###yes.###speakerb93.###how did you hear,###oh we're not even supposed *-1 to be talking about this though, are we *?*?###speakera94.###i guess not.###speakerb95.###speakera96."
256,4270:11,"so, and how do you feel about it *t*-1?.","###yeah,###so. i'm, i'm an avid aerobics, uh, person and just very, very athletic.###speakera9.###uh-huh.###speakerb10.###enjoying a th-, you know, uh, anything dealing with sports.###speakera11.###uh-huh,###uh-huh.###speakerb12."


In [27]:
who["AResponse"] = "Who is a person...?"
# who["SomeResponse"] = "Who is some person...?"
who["AllResponse"] = "Who is every person...?"
who["TheResponse"] = "Who is the person...?"


where["AResponse"] = "What is a place...?"
# where["SomeResponse"] = "What is some place...?"
where["AllResponse"] = "What is every place...?"
where["TheResponse"] = "What is the place...?"


how["AResponse"] = "What is a way...?"
# how["SomeResponse"] = "What is some way...?"
how["AllResponse"] = "What is every way...?"
how["TheResponse"] = "What is the way...?"

<ipython-input-27-db43c0f19116>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  who["AResponse"] = "Who is a person...?"
<ipython-input-27-db43c0f19116>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  who["AllResponse"] = "Who is every person...?"
<ipython-input-27-db43c0f19116>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [83]:
df_final = pd.concat([who,where,how])

In [88]:
len(df_final)

357

In [29]:
df_final.head()

,TGrepID,EntireSentence,PreceedingContext,AResponse,AllResponse,TheResponse
274,4787:18,"and, you know, who *t*-1's *-2 to put the limit on it?","###speakerb38.###you know.###speakera39.###yeah.###speakerb40.###they can't be accepted *-1 into society.###you know, if they're not going *-1 to be accepted *-2 into society, then everybody's going *-3 to become sociopathic.###speakera41.###huh-uh.###speakerb42.",Who is a person...?,Who is every person...?,Who is the person...?
307,5433:12,i mean who would *t*-1 steal a newspaper?,"###speakera47.###uh-huh.###i find it *exp*-1 hard * to, uh, follow from one page to another.###it's just something 0 i have never developed, uh, an interest in *t*-1.###speakerb48.###huh.###speakera49.###and i live in kind of a bad area where if i have the paper delivered *-1 *t*-2 it's stolen *-3 before i can get out and get it.###speakerb50.###gosh that's, that sounds pretty bad to me.",Who is a person...?,Who is every person...?,Who is the person...?
309,5449:4,who would *t*-1 steal a newspaper?,"###they just come by and pick them up even if it's just for the tv, you know t-, uh, selections of the day.###speakerb54.###i see.###speakera55.###i don't know.###it just wouldn't be there whenever i tried it *t*-1.###speakerb56.###yeah.###speakera57.###so, you're right,",Who is a person...?,Who is every person...?,Who is the person...?
409,6696:53,"what would you have, who, who, who would *t*-2 determine what these people do *t*-1.","###yeah.###speakerb72.###i don't know.###speakera73.###yeah###the only,###speakerb74.###like * to have a volunteer come here and rake leaves and mow the grass and,###speakera75.###well, you know that brings up the interesting subject too, you know,",Who is a person...?,Who is every person...?,Who is the person...?
415,6797:4,"who *t*-1's going *-2 to run it, you know,","###speakera109.###yeah.###speakerb110.###speakera111.###yeah,### it's, it's, it's just the, you know,###here you are *t*-1 to the logistics of the thing again, you know.###speakerb112.###yeah.###speakera113.",Who is a person...?,Who is every person...?,Who is the person...?


There should be a way to automate that whole thing in one definitieion, but im failing to do that correctly

In [19]:
# def paramap(data):
#     if data["EntireSentence"].str.contains("who|Who"):
#         data["AResponse"] == "Who is a person?"
#         data["SomeResponse"] == "Who is some person?"
#         data["AllResponse"] == "Who is every person?"
#         data["TheResponse"] == "Who is the person?"
    #         return
    #     if df["EntireSentence"].str.contains(["where|Where"]):
    #         return "places"
    #     if df["EntireSentence"].str.contains(["how|How"]):
    #         return "ways"

In [20]:
# df_para = df_valid.apply(paramap)

## Create randomly sampled files

## Pilot Samples

### Pilot Ex 01

In [21]:
pilot = df_final.sample(10, random_state=333)

In [22]:
# pilot.to_csv("../../experiments/clean_corpus/pilot1.txt",header=True,sep="\t",index=False)

### Pilot Ex 02
This design uses sliders rather than radio buttons, and is not a forced choice.

Stimuli are the same as Pilot Ex01, but with rhetorical questions replaced.

In [23]:
# rhetorical questions to remove
# 5928, 2328
pilot.EntireSentence

3582                                                    how are your kids handling it *t*-1?
3511                                                                        who *t*-1 knows.
2178                           well how do you think 0 the royals are going *-1 to do *t*-2?
5928    but, uh, who *t*-1 knows what the future will hold *t*-2. uh, as far as energy goes,
5859                                      how do you usually find a, a new restaurant *t*-1?
2931                                                           uh, who's the last one *t*-1.
7463                          oh, g-, where *ich*-3 have i gone *t*-1 that *t*-2's anywhere.
2328        you know, how can you help *-1 depending on the day care center *t*-2, you know,
6779                                                  uh, how do you feel about taxes *t*-1?
3938                                                                  where do you go *t*-1?
Name: EntireSentence, dtype: object

In [24]:
sample2 = df_final.sample(2, random_state=33)

In [25]:
sample2.PreceedingContext

5347    ###that's all mexico, i think 0 *t*-1.###speakerb57.###yes,###uh-huh.### and, but it was wonderful.###so we really like those.###and, uh, i guess 0 we just like *-1 to go someplace, someplace entirely different.###we've been to a few places in this country.###we still have a lot 0 *-1 to see *t*-2, though.###speakera58.
7270                                                                                                              ###i mean, pretty much it's a white shirt and slacks and shoes and a coat.###speakerb12.###uh-huh.###speakera13.###speakerb14.###uh-huh.###speakera15.###speakerb16.###speakera17.###but, it was very different before.
Name: PreceedingContext, dtype: object

In [26]:
sample2.EntireSentence

5347    where are you from *t*-1 originally?
7270                 how do you dress *t*-1?
Name: EntireSentence, dtype: object

In [27]:
pilot = pilot.drop([5928,2328], axis=0)

In [32]:
pilot2 = pd.concat([pilot, sample2])

In [36]:
# remove the someresponse column
pilot2 = pilot2.drop(columns="SomeResponse")

In [37]:
pilot2.head(1)

,TGrepID,EntireSentence,PreceedingContext,AResponse,AllResponse,TheResponse
3582,61578:4,how are your kids handling it *t*-1?,"###speakera161.###no,### i, i know,###i mean he's always, you know, he really does have a good home.###i think 0 that's what *t*-1 really bothers me the most about it.###speakerb162.###yeah.###speakera163.###you know, it's like,###speakerb164.",What is a way...?,What is every way...?,What is the way...?


In [38]:
pilot2.to_csv("../../experiments/clean_corpus/pilot2.txt",header=True,sep="\t",index=False)

### TEST SAMPLE

deseridata:
15 how , 10 where, 5 who

should contain about 3 modal in each batch
10 lists of 30
1 list of 35


In [70]:
len(df_final)

357

In [74]:
357/30

11.9

In [75]:
len(df_final)

357

In [80]:
df_final.sample(5, replace=True, random_state=97098098)

,TGrepID,EntireSentence,PreceedingContext,AResponse,AllResponse,TheResponse
2346,40579:20,"where do, where do you live *t*-1?","###right.###but, i wasn't embarrassed * to shop around you know,###speakerb124.###no.###speakera125.###so, the, the person 0 we found it through *t*-1 the,###there's an association *ich*-1 here in town of professional home, care providers or something like that.###speakerb126.###speakera127.###speakerb128.",What is a place...?,What is every place...?,What is the place...?
10159,177088:4,how can you really deal with an environment where you're going *-1 to lose everything *t*-2 *t*-3.,"###speakerb26.### t-, they're only going *-1 to suffer.### they, they've defeated themselves with, with their looting and violence.###uh, i understand why they were upset *-2 by the verdict *t*-3 just because even those of us that *t*-1 aren't black or, uh, don't live there or whatever, we all saw the same thing on the, on the tape###and, uh,###speakera27.###well e-, everybody is saying 0 this is going *-1 to be a new era###and we're going *-1 to reexamine this thing.###hey, the b-, the average person is going *-1 to take a look at that and say, suspicions confirmed.###they're a bunch of animals, you know.",What is a way...?,What is every way...?,What is the way...?
7071,122929:4,how would you achieve that *t*-1?,"###speakera9.###yes.###speakerb10.###speakera11.###uh-huh.###speakerb12.###uh, we need *-1 to elect people that *t*-2 are more, uh,###or...###speakera13.###i'm not sure how you would ach-,",What is a way...?,What is every way...?,What is the way...?
9533,168089:7,now where does the source light go *t*-1?,"###what they do *t*-1, they do several different types of things.###sometimes they put a source light###and they look at the source light###and then as the car goes by, you can see the exhaust pipe and the source light behind it###and you can look at the emissions actually.###speakerb31.###huh.###speakera32.###and...###speakerb33.",What is a place...?,What is every place...?,What is the place...?
7759,136873:4,"how can you watch that *t*-1, you know","###speakerb106.###yeah,###i don't watch very much of it at all, e-, to, to the extent that sometimes i feel almost embarrassed at work and stuff like that###yeah,###speakera107.###because they talk so much,###isn't that the,###speakerb108.###they talk about t v shows,###and i, huh, you know,",What is a way...?,What is every way...?,What is the way...?


In [81]:
len(df_final)

357

In [72]:
def generate_stimuli(df): 
    for i in range(1,13):
        if i < 13: 
            who = df[df["EntireSentence"].str.contains("Who|who")]
            where = df[df["EntireSentence"].str.contains("Where|where")]
            how = df[df["EntireSentence"].str.contains("How|how")]

            who_sample = who.sample(5, replace=False, random_state=i)
            where_sample = where.sample(10, replace=False, random_state=i)
            how_sample = how.sample(15, replace=False, random_state=i)

            l = pd.concat([who_sample,where_sample,how_sample])
            print(len(l))
#             print(len(who))
#             print(len(where))
#             print(len(how))
    #         print("remaining who: {}".format(n=len(who_sample)))
    #         print("remaining where: {}".format(n=len(where_sample)))
    #         print("remaining how: {}".format(n=len(how_sample)))
            filename = f"../../experiments/clean_corpus/corpus_{i}.txt".format(i=i)
            l.to_csv(filename,header=True,sep="\t",index=False)
        else:
            final = pd.concat([who,where,how])
            len(final)
            filename = f"../../experiments/clean_corpus/corpus_{i}.txt".format(i=i)
            final.to_csv(filename,header=True,sep="\t",index=False)
    

In [73]:
generate_stimuli(df_final)

30
30
30
30
30
30
30
30
30
30
30
30


In [68]:
11*30

330

In [91]:
# get the columns with 'wh' and 'modalpresent'
df_cols = df[df["Item_ID"].isin(set(crit_index))]

In [92]:
df_cols.head

<bound method NDFrame.head of          Item_ID  \
95        1721:4   
189      2936:24   
246       4134:4   
247       4140:4   
256      4270:11   
274      4787:18   
302      5370:44   
307      5433:12   
309       5449:4   
313       5532:4   
316      5604:48   
318       5622:4   
319       5652:4   
320      5673:22   
408      6696:53   
414       6797:4   
455      7652:23   
508       8552:6   
511       8565:4   
577       9885:4   
623      11053:4   
640      11493:7   
644      11550:4   
745     13335:49   
808      14025:4   
809     14041:20   
825      14266:4   
833     14478:20   
874      15049:9   
909      15722:4   
940      16417:9   
1001     17923:4   
1027     18367:4   
1051     18885:4   
1077    19308:21   
1114     19768:4   
1129    20121:21   
1143     20333:4   
1227    21649:34   
1233     21725:6   
1304     22805:4   
1319     23105:4   
1330     23378:4   
1394    24525:28   
1398    24572:42   
1401     24617:9   
1417    24955:16   
1418     2

In [31]:
d.to_csv("testfile.txt",header=True,sep="\t",index=False)

NameError: name 'd' is not defined